# No-glue-code

In [ ]:
using Pkg
Pkg.activate("..")

In [ ]:
# The statistical inference frame-work we will use
using LogDensityProblems
using LogDensityProblemsAD
using DynamicPPL
using ForwardDiff
using Random
using LinearAlgebra

#Plotting
using PyPlot

#What we are tweaking
using Revise
using AdvancedHMC
using Turing

## Model

In [ ]:
# Just a simple Neal Funnel
d = 21
@model function funnel()
    θ ~ Normal(0, 3)
    z ~ MvNormal(zeros(d-1), exp(θ)*I)
    x ~ MvNormal(z, I)
end

In [ ]:
Random.seed!(1)
(;x) = rand(funnel() | (θ=0,))
funnel_model = funnel() | (;x)

## Turing interface

In [ ]:
struct Sampler
    metric
    integrator
    adaptor
    proposal
end

Sampler(ϵ::Number, TAP::Number) = begin
    metric =  DiagEuclideanMetric(d)
    integrator = Leapfrog(ϵ)
    proposal = AdvancedHMC.NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(TAP, integrator))
    
    Sampler(
        metric,
        integrator,
        adaptor,
        proposal)
end

In [ ]:
initial_θ = randn(21)
initial_ϵ = 0.1 #find_good_stepsize(hamiltonian, initial_θ)
spl = Sampler(initial_ϵ, 0.95)

## Sampling

In [ ]:
n_samples, n_adapts = 10_000, 1_000
samples, stats = sample(funnel_model, 0.1, 0.95, n_samples, n_adapts; initial_θ=initial_θ);

## Plotting

In [ ]:
theta_mchmc = [sample[1] for sample in samples]
x10_mchmc = [sample[10+1] for sample in samples];

In [ ]:
fig, axis = plt.subplots(2, 2, figsize=(8,8))
fig.suptitle("Neal's Funnel", fontsize=16)

fig.delaxes(axis[1,2])
fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0)

axis[1,1].hist(x10_mchmc, bins=100, range=[-6,2])
axis[1,1].set_yticks([])

axis[2,2].hist(theta_mchmc, bins=100, orientation="horizontal", range=[-4, 2])
axis[2,2].set_xticks([])
axis[2,2].set_yticks([])

axis[2,1].hist2d(x10_mchmc, theta_mchmc, bins=100, range=[[-6,2],[-4, 2]])
axis[2,1].set_xlabel("x10")
axis[2,1].set_ylabel("theta");

## Sampling w AbstractMCMC

In [ ]:
initial_ϵ=0.1 
TAP=0.95
ss = AdvancedHMC.HMCSamplerSettings(initial_ϵ, TAP)

In [ ]:
sample(funnel_model, ss, 1000)

## Sampling w Turing

In [ ]:
using Turing

In [ ]:
TAP = 0.95
nadapts = 300
spl = Turing.NUTS(nadapts, TAP)

In [ ]:
Turing.sample(funnel_model, spl, 50_000, progress=true; save_state=true)